<a href="https://colab.research.google.com/github/rohanBrid18/ML-Workshop/blob/master/Transfer_Learning_with_Keras.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import os
import keras
import matplotlib.pyplot as plt
from keras.layers import Dense,GlobalAveragePooling2D
from keras.applications import MobileNet
from keras.preprocessing import image
from keras.applications.mobilenet import preprocess_input
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Model
from keras.optimizers import Adam

Using TensorFlow backend.


In [2]:
base_model=MobileNet(weights='imagenet',include_top=False) #imports the mobilenet model and discards the last 1000 neuron layer.

x=base_model.output
x=GlobalAveragePooling2D()(x)
x=Dense(1024,activation='relu')(x) #we add dense layers so that the model can learn more complex functions and classify for better results.
x=Dense(1024,activation='relu')(x) #dense layer 2
x=Dense(512,activation='relu')(x) #dense layer 3
preds=Dense(5,activation='softmax')(x) #final layer with softmax activation

/usr/local/lib/python3.6/dist-packages/keras_applications/mobilenet.py:207: UserWarning: `input_shape` is undefined or non-square, or `rows` is not in [128, 160, 192, 224]. Weights for input shape (224, 224) will be loaded as the default.
  warnings.warn('`input_shape` is undefined or non-square, '
W0627 18:18:44.258430 140464382293888 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0627 18:18:44.294910 140464382293888 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0627 18:18:44.304259 140464382293888 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.un

17227776/17225924 [==============================] - 1s 0us/step


In [0]:
# make a model based on the architecture we have provided
model=Model(inputs=base_model.input,outputs=preds)

In [4]:
# check the architecture of our model
for i,layer in enumerate(model.layers):
  print(i,layer.name)

0 input_1
1 conv1_pad
2 conv1
3 conv1_bn
4 conv1_relu
5 conv_dw_1
6 conv_dw_1_bn
7 conv_dw_1_relu
8 conv_pw_1
9 conv_pw_1_bn
10 conv_pw_1_relu
11 conv_pad_2
12 conv_dw_2
13 conv_dw_2_bn
14 conv_dw_2_relu
15 conv_pw_2
16 conv_pw_2_bn
17 conv_pw_2_relu
18 conv_dw_3
19 conv_dw_3_bn
20 conv_dw_3_relu
21 conv_pw_3
22 conv_pw_3_bn
23 conv_pw_3_relu
24 conv_pad_4
25 conv_dw_4
26 conv_dw_4_bn
27 conv_dw_4_relu
28 conv_pw_4
29 conv_pw_4_bn
30 conv_pw_4_relu
31 conv_dw_5
32 conv_dw_5_bn
33 conv_dw_5_relu
34 conv_pw_5
35 conv_pw_5_bn
36 conv_pw_5_relu
37 conv_pad_6
38 conv_dw_6
39 conv_dw_6_bn
40 conv_dw_6_relu
41 conv_pw_6
42 conv_pw_6_bn
43 conv_pw_6_relu
44 conv_dw_7
45 conv_dw_7_bn
46 conv_dw_7_relu
47 conv_pw_7
48 conv_pw_7_bn
49 conv_pw_7_relu
50 conv_dw_8
51 conv_dw_8_bn
52 conv_dw_8_relu
53 conv_pw_8
54 conv_pw_8_bn
55 conv_pw_8_relu
56 conv_dw_9
57 conv_dw_9_bn
58 conv_dw_9_relu
59 conv_pw_9
60 conv_pw_9_bn
61 conv_pw_9_relu
62 conv_dw_10
63 conv_dw_10_bn
64 conv_dw_10_relu
65 conv_pw_10

In [0]:
for layer in model.layers[:20]:
    layer.trainable=False
for layer in model.layers[20:]:
    layer.trainable=True

In [6]:
!ls -ltr

total 4
drwxr-xr-x 1 root root 4096 Jun 18 16:14 sample_data


In [7]:
# download the flower photos dataset
%%time

from six.moves import urllib

FLOWERS_DIR = './flower_photos'

if not os.path.exists(FLOWERS_DIR):
  DOWNLOAD_URL = 'http://download.tensorflow.org/example_images/flower_photos.tgz'
  print('Downloading flower images from %s...' % DOWNLOAD_URL)
  urllib.request.urlretrieve(DOWNLOAD_URL, 'flower_photos.tgz')
  !tar xfz flower_photos.tgz
print('Flower photos are located in %s' % FLOWERS_DIR)

Flower photos are located in ./flower_photos
CPU times: user 198 ms, sys: 403 ms, total: 602 ms
Wall time: 10.2 s


In [8]:
!ls -ltr

total 223460
drwxr-x--- 7 270850 5000      4096 Feb 10  2016 flower_photos
drwxr-xr-x 1 root   root      4096 Jun 18 16:14 sample_data
-rw-r--r-- 1 root   root 228813984 Jun 27 18:19 flower_photos.tgz


In [9]:
# generate training data
train_datagen=ImageDataGenerator(preprocessing_function=preprocess_input) #included in our dependencies

train_generator=train_datagen.flow_from_directory('./flower_photos/', # this is where you specify the path to the main data folder
                                                 target_size=(224,224),
                                                 color_mode='rgb',
                                                 batch_size=32,
                                                 class_mode='categorical',
                                                 shuffle=True)

Found 3670 images belonging to 5 classes.


In [10]:
model.compile(optimizer='Adam',loss='categorical_crossentropy',metrics=['accuracy'])
# Adam optimizer
# loss function will be categorical cross entropy
# evaluation metric will be accuracy

step_size_train=train_generator.n//train_generator.batch_size
model.fit_generator(generator=train_generator,
                   steps_per_epoch=step_size_train,
                   epochs=10)

W0627 18:20:50.334061 140464382293888 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.

W0627 18:20:50.463307 140464382293888 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Epoch 1/10
114/114 [==============================] - 20s 179ms/step - loss: 0.5983 - acc: 0.7966
Epoch 2/10
114/114 [==============================] - 14s 124ms/step - loss: 0.3660 - acc: 0.8868
Epoch 3/10
114/114 [==============================] - 14s 123ms/step - loss: 0.2902 - acc: 0.9067
Epoch 4/10
114/114 [==============================] - 14s 123ms/step - loss: 0.2792 - acc: 0.9093
Epoch 5/10
114/114 [==============================] - 14s 124ms/step - loss: 0.2209 - acc: 0.9284
Epoch 6/10
114/114 [==============================] - 14s 124ms/step - loss: 0.1930 - acc: 0.9436
Epoch 7/10
114/114 [==============================] - 14s 124ms/step - loss: 0.1623 - acc: 0.9538
Epoch 8/10
114/114 [==============================] - 14s 123ms/step - loss: 0.1672 - acc: 0.9573
Epoch 9/10
114/114 [==============================] - 14s 123ms/step - loss: 0.1541 - acc: 0.9545
Epoch 10/10
114/114 [==============================] - 14s 124ms/step - loss: 0.1170 - acc: 0.9691
